# Baixando os pré-requisitos

In [ ]:
!pip install pdfplumber
import pdfplumber

import nltk 
nltk.download('stopwords')

import pandas as pd
import numpy as np


# Configurando a leitura do arquivo de vagas

In [ ]:
vagas = pd.read_excel('vagas.xlsx', sheet_name = None)
n_vagas = len(vagas.keys())
nome_vagas = list(vagas.keys())
vagas = [vagas[nome_vagas[i]] for i in range(n_vagas)]

# Declarando a função da aplicação

In [ ]:
def CVMatch(cv, vaga, limite = 5):

    arquivoPDF = pdfplumber.open(cv)
    primeira_pagina = arquivoPDF.pages[0]
    textoCRU = primeira_pagina.extract_text()

    lista_de_palavras = nltk.tokenize.word_tokenize(textoCRU)
    lista_de_palavras = [palavra.lower() for palavra in lista_de_palavras]
    pontuacao = ['(',')',';',':','[',']',',','–','(',')','/','|','-','%','@']
    stop_words = nltk.corpus.stopwords.words('portuguese')
    keywords = [palavra for palavra in lista_de_palavras if not palavra in stop_words and not palavra in pontuacao]
    textocv = " ".join(s for s in keywords)

    pesos = list(vaga['pesos'])
    palavras_chaves = list(vaga['palavras-chave'])

    cont = [textocv.count(pc) for pc in palavras_chaves]

    def aux(x, limite):
        return x if x <= limite else limite

    cont = [aux(i, limite) for i in cont]

    pmax = np.sum(np.array(pesos) * limite) 

    score = ((np.array(cont) * pesos).sum()/pmax).round(2)

    return score

# Inserindo curriculos e obtendo a pontuação

In [ ]:
curriculo = ['/caminho/arquivo1.pdf',
             '/caminho/arquivo2.pdf']
pessoas = [[CVMatch(cv, vaga) for vaga in vagas] for cv in curriculo]
nomes = [cv.split('/')[-1].split('.')[0] for cv in curriculo]
matchs = pd.DataFrame(pessoas, columns = nome_vagas, index = nomes)
matchs.sort_values(by = 'dev_back', ascending = False)
